# Lab 3: Stochastic Gradient Descent

Welcome to the advanced Machine Learning Course.

The objective of this lab session is to code a few regression algorithms and to apply them to synthetic and real datasets.

Please put **"ML - MDS - TD3"** in the mail subject or I might lose your work (which means 0) and send it to pierre.houdouin@centralesupelec.fr

Please label your notebook **"L3_familyname1_familyname2.ipynb"** or I might lose your work (which means 0).

We begin with the standard imports:

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
from tqdm import tqdm

We import the dataset that we are going to use, an indian dataset including in the last column information about the diabetes status of patients:

In [5]:
from sklearn import model_selection

data = pd.read_csv("w8a.csv", sep=";", header=None)

X = data.iloc[:,:-1].to_numpy()
y = data.iloc[:,-1].to_numpy()
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.33, random_state=32)

## Logistic Regression



Today we'll be moving from linear regression to logistic regression, one of the simplest ways to deal with a classification problem. Instead of fitting a line, logistic regression models the probability that the outcome is 1 given the value of the predictor. In order to do this we need a function that transforms our predictor variable to a value between 0 and 1. Lots of functions can do that, but the logistic function is the most common choice:

$$f(z) = \frac{1}{1+\exp{-z}}.$$

To predict the class of our observations we'll have to minimize the corresponding loss function and as we are in a high-dimensional context we'll add an $l_2$ regularization to the model:

$$L(\textbf{w}) = \sum_{i=1}^n log(1+\exp(-y_i\textbf{w}^Tx_i))+\frac{\lambda}{2} \| \textbf{w} \|^2,$$

where $x_i$ is the vector of features for the observation $i$ and $y_i \in \{-1, 1\}$ is the class label.  


We first use the `sklearn` implementation:

In [6]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=32, max_iter=1000)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

and we compute the accuracy score to evaluate the model performance:

In [7]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9838


### Assignment

Implement from scratch your own logistic regression model with stochastic gradient descent optimization. 

- Fill in the class

- Display the evolution of the cost function along iterations. Do this for several strategies for the setting of the learning rate

- Try the different acceleration strategies

- Train the model with the training set and evaluate its performance in the test set

In [11]:
from sklearn.utils import shuffle

class StochasticLogisticRegression():
    
    """ Class for logistic regression:
    
    Attributes:                                                                                 
    -----------                                                                              Default value
    coef_         : 1-dimensional np.array, coefficients / weights                         | None                
    lambd_        : float,                  regularization parameter                       | 0.1
    lr_           : float,                  the learning rate                              | 0.01
    bsize         : integer,                the size of the mini-batch >=1                 | 64
    gamma         : float,                  gamma coefficient                              | 0.999
    beta          : float,                  beta coefficient                               | 0.9
    eps           : float,                  epsilon coefficient                            | 10-8
    debias        : boolean,                indicates if we use the debiais correction     | False
    coef_history_ : list,                   the list of all visited betas/ weights         | []
    f_history_    : list ,                  the list of all evaluations in visited betas   | []
    thresh        : float,                  decision threshold for classification          | 0.5
    """
    
    def __init__(self, lambd=0.1, lr=1e-2, batchsize=64, gamma=0.999, beta=0.9, eps=1e-8, debias=False,
                 thresh=0.5):
        self.coef_         = None # weights
        self.lambd_        = lambd
        self.lr_           = lr
        self.bsize_        = batchsize
        self.gamma_        = gamma
        self.beta_         = beta
        self.eps_          = eps
        self.debias_       = debias
        self.coef_history_ = []
        self.f_history_    = []
        self.thresh_       = thresh

    def logistic(self, z):
        return 1 / (1 + np.exp(-z))
    
    def loss(self, X, y):
        z = X.dot(self.coef_)
        return np.mean(np.log(1 + np.exp(-y * z))) + 0.5 * self.lambd_ * np.sum(self.coef_ ** 2)
        
    def fit(self, X, y, lr=1e-2, bsize=64, max_iter=100, minibatch=False):
        
        """ Fit the data (X, y).
    
        Parameters:
        -----------                                                                         Default value
        X          : (num_samples, num_features) np.array, Design matrix                  | 
        y          : (num_sampes, ) np.array,              Output vector                  | 
        lr         : float,                                the learning rate              | 0.001
        bsize      : integer,                              the size of the mini-batch >=1 | 64
        max_iter   : integer,                              the number of epochs           | 100
        mini_batch : bool,                                 method used                    | False
        """
        n_samples, n_features = X.shape
        self.coef_ = np.zeros(n_features)
        m = np.zeros_like(self.coef_)
        v = np.zeros_like(self.coef_)
        t = 0

        for epoch in range(max_iter):
            if minibatch:
                X, y = shuffle(X, y)
                for i in range(0, n_samples, bsize):
                    X_batch = X[i:i+bsize]
                    y_batch = y[i:i+bsize]
                    
                    z = X_batch.dot(self.coef_)
                    h = self.logistic(z)
                    grad = X_batch.T.dot(h - y_batch) / bsize + self.lambd_ * self.coef_
                    
                    t += 1
                    m = self.beta_ * m + (1 - self.beta_) * grad
                    v = self.gamma_ * v + (1 - self.gamma_) * (grad ** 2)
                    
                    if self.debias_:
                        m_hat = m / (1 - self.beta_ ** t)
                        v_hat = v / (1 - self.gamma_ ** t)
                    else:
                        m_hat, v_hat = m, v
                    
                    self.coef_ -= lr * m_hat / (np.sqrt(v_hat) + self.eps_)
            else:
                z = X.dot(self.coef_)
                h = self.logistic(z)
                grad = X.T.dot(h - y) / n_samples + self.lambd_ * self.coef_
                
                t += 1
                m = self.beta_ * m + (1 - self.beta_) * grad
                v = self.gamma_ * v + (1 - self.gamma_) * (grad ** 2)
                
                if self.debias_:
                    m_hat = m / (1 - self.beta_ ** t)
                    v_hat = v / (1 - self.gamma_ ** t)
                else:
                    m_hat, v_hat = m, v
                
                self.coef_ -= lr * m_hat / (np.sqrt(v_hat) + self.eps_)
            
            self.coef_history_.append(self.coef_.copy())
            self.f_history_.append(self.loss(X, y))

        

    def predict(self, X):
        """ Make binary predictions for data X.
    
        Parameters:
        -----------
        X: (num_samples, num_features) np.array, Design matrix
        
        Returns:
        -----
        y_pred: (num_samples, ) np.array, Predictions (0 or 1)
        """
        probas = self.logistic(X.dot(self.coef_))
        return (probas >= self.thresh_).astype(int)
    
    def accuracy_evaluation(self,X,y,thresh=0.5,margin=0, vizualisation=True):
        """ Compute detailed accuracy rates.
    
        Parameters:
        -----------                                                              Default value
        X                : (num_samples, num_features) np.array, Input data    | 
        y                : (num_samples, ) np.array, Labels of the input data  | 
        thresh           : Decision threshold                                  | 0.5
        margin           : If the prediction is at a distance less than margin | 0
                           to the treshold, returns undetermined label.        |
        vizualisation    : Allow the user to vizualize in a board the results  | True
            
        Returns:
        -----
        good_prediction  : % of correct classifications
        undetermined     : % of indetermined labels 
        wrong_prediction : % of wrong classifications
        TP               : % of 1 labelled 1
        UP               : % of 1 labelled undetermined
        FN               : % of 1 labelled 0
        TN               : % of 0 labelled 0
        UN               : % of 0 labelled undetermined
        FP               : % of 0 labelled 1
        F1_score         : F1-score
        """  
        probas = self.logistic(X.dot(self.coef_))
        y_pred = np.zeros_like(y)
        y_pred[probas > thresh + margin] = 1
        y_pred[probas < thresh - margin] = 0
        y_pred[(probas >= thresh - margin) & (probas <= thresh + margin)] = -1  # Undetermined

        TP = np.sum((y == 1) & (y_pred == 1)) / np.sum(y == 1)
        UP = np.sum((y == 1) & (y_pred == -1)) / np.sum(y == 1)
        FN = np.sum((y == 1) & (y_pred == 0)) / np.sum(y == 1)
        TN = np.sum((y == 0) & (y_pred == 0)) / np.sum(y == 0)
        UN = np.sum((y == 0) & (y_pred == -1)) / np.sum(y == 0)
        FP = np.sum((y == 0) & (y_pred == 1)) / np.sum(y == 0)

        good_prediction = np.mean(y == y_pred)
        undetermined = np.mean(y_pred == -1)
        wrong_prediction = np.mean((y != y_pred) & (y_pred != -1))

        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        F1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        if vizualisation:
            print(f"Good predictions: {good_prediction:.2%}")
            print(f"Undetermined: {undetermined:.2%}")
            print(f"Wrong predictions: {wrong_prediction:.2%}")
            print(f"True Positives: {TP:.2%}")
            print(f"Undetermined Positives: {UP:.2%}")
            print(f"False Negatives: {FN:.2%}")
            print(f"True Negatives: {TN:.2%}")
            print(f"Undetermined Negatives: {UN:.2%}")
            print(f"False Positives: {FP:.2%}")
            print(f"F1-score: {F1_score:.4f}")

        return good_prediction, undetermined, wrong_prediction, TP, UP, FN, TN, UN, FP, F1_score

    
    def find_thresh(self, X, y, step=0.01,margin=0):
        """Find the decision threshsold that maximize the f1_score
        
        Parameters:
        -----------                                                    Default value
        X      : (num_samples, num_features) np.array, Input data    | 
        y      : (num_samples, ) np.array, Labels of the input data  | 
        step   : Decision threshold                                  | 0.5
        margin : If the prediction is at a distance less than margin | 0
                 to the treshold, returns undetermined label.        |
        """  
        best_thresh = 0
        best_f1 = 0
        for thresh in np.arange(0, 1 + step, step):
            _, _, _, _, _, _, _, _, _, f1_score = self.accuracy_evaluation(X, y, thresh, margin, visualization=False)
            if f1_score > best_f1:
                best_f1 = f1_score
                best_thresh = thresh
        self.thresh_ = best_thresh
        return best_thresh, best_f1

Apply to the data

On essaie plusieurs entraînements avec différents coefficients de régularisation, on choisit celui qui minimise
l'erreur sur le test set. On entraine sur 1000 itérations en batch gradient descent. On garde **lambda = 2**.
On essaie ensuite plusieurs learning rates, on choisit le plus grand qui permet d'obtenir un gradient qui converge. On garde **lr = 0.01**.

## Optimal threshold

In [13]:
model = StochasticLogisticRegression(2, 0.01, 64)
model.fit(X_train, y_train, max_iter=1000)

# Make predictions on the test set
y_pred = model.predict(X_test)

model.accuracy_evaluation(X_test, y_test)

# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy:.4f}")

Good predictions: 8.36%
Undetermined: 8.65%
Wrong predictions: 91.29%
True Positives: 1.87%
Undetermined Positives: 10.86%
False Negatives: 87.27%
True Negatives: nan%
Undetermined Negatives: nan%
False Positives: nan%
F1-score: 0.0000
Accuracy: 0.0041


C:\Users\bouet\AppData\Local\Temp\ipykernel_31428\652998116.py:155: RuntimeWarning: invalid value encountered in scalar divide
  TN = np.sum((y == 0) & (y_pred == 0)) / np.sum(y == 0)
C:\Users\bouet\AppData\Local\Temp\ipykernel_31428\652998116.py:156: RuntimeWarning: invalid value encountered in scalar divide
  UN = np.sum((y == 0) & (y_pred == -1)) / np.sum(y == 0)
C:\Users\bouet\AppData\Local\Temp\ipykernel_31428\652998116.py:157: RuntimeWarning: invalid value encountered in scalar divide
  FP = np.sum((y == 0) & (y_pred == 1)) / np.sum(y == 0)


## Training of the model

In [2]:
#TODO

Implement only one acceleration method and compare the results

## Study of the batchsize impact

In [3]:
#TODO : Study the loss

In [4]:
#TODO : Study the accuracy

## Gamma impact

In [5]:
#TODO : Study Loss and F1-score

## Beta impact
Nous allons comparer les paramètres beta pour gamma = 0.999 et batch_size=1024

In [6]:
#TODO : Study Loss and F1-score